In [4]:
!pip install spacy


In [7]:
!pip install flask

In [8]:
!pip install spacy-transformers  

   ---------------------------------------- 0.0/8.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.2 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.2 MB 204.5 kB/s eta 0:00:38
   -- ------------------------------------- 0.5/8.2 MB 204.5 kB/s eta 0:00:38
   -- ------------------------------------- 0.5/8.2 MB 204.5 kB/s eta 0:00:38
   -- ------------------------------------- 0.5/8.2 MB 204.5 kB/s eta 0:00:3

In [2]:
!pip install en_core_web_md-3.5.0.tar.gz


Processing c:\users\wajiz.pk\desktop\ai intern task\chatbot\en_core_web_md-3.5.0.tar.gz
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for en_core_web_md: filename=en_core_web_md-3.5.0-py3-none-any.whl size=42778850 sha256=25d58a444b47b86137ba8534edf7684ca5c676f32144792136a22c2b0d334abf
  Stored in directory: c:\users\wajiz.pk\appdata\local\pip\cache\wheels\19\7a\af\675ecb34b4e23ab687316cb25676d396303fca2a70862f2076
Successfully built en_core_web_md
  Attempting uninstall: en_core_web_md
    Found existing installation: en_core_web_md 3.5.0
    Uninstalling en_core_web_md-3.5.0:
      Successfully uninstalled en_core_web_md-3.5.0


Load spaCy and Preprocess FAQs

In [3]:
import spacy
import json

# Load spaCy model
nlp = spacy.load("en_core_web_md")  # Use a medium model for better accuracy

# Load FAQs
with open("faqs.json", "r") as f:
    faqs = json.load(f)

# Preprocess FAQs for semantic similarity
def preprocess_faqs(faqs):
    return [(faq['question'], nlp(faq['question'])) for faq in faqs]

faq_data = preprocess_faqs(faqs)


TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

Create a Function for Matching Questions

In [26]:
def get_best_match(user_query, faq_data, threshold=0.7):
    query_doc = nlp(user_query)
    best_match = None
    best_score = 0

    for question, question_doc in faq_data:
        similarity = query_doc.similarity(question_doc)
        if similarity > best_score:
            best_match = question
            best_score = similarity

    if best_score > threshold:
        return best_match
    else:
        return "Sorry, I couldn't find an answer to your question."


Integrate with Flask for a Simple Web Chatbot

In [41]:
from flask import Flask, request, jsonify
import json
from difflib import get_close_matches
import os

app = Flask(__name__)

def load_faq_data():
    """Load FAQ data from JSON file with error handling"""
    try:
        # Check if file exists
        if not os.path.exists("faqs.json"):
            # Create default FAQ data if file doesn't exist
            default_faqs = [
                {
                    "question": "What is this?",
                    "answer": "This is a FAQ API service."
                },
                {
                    "question": "How does it work?",
                    "answer": "It matches your question to the closest FAQ answer."
                }
            ]
            # Save default FAQs
            with open("faqs.json", "w") as f:
                json.dump(default_faqs, f, indent=4)
            return default_faqs
            
        with open("faqs.json", "r", encoding='utf-8') as f:
            data = json.load(f)
            if not isinstance(data, list):
                raise ValueError("FAQ data must be a list of questions and answers")
            return data
    except json.JSONDecodeError:
        print("Error: faqs.json is not valid JSON")
        return []
    except Exception as e:
        print(f"Error loading FAQ data: {str(e)}")
        return []

# Load FAQs when starting the app
faq_data = load_faq_data()

def get_best_match(user_input, faq_data):
    """Find the best matching question with error handling"""
    try:
        if not user_input or not isinstance(user_input, str):
            return None
            
        questions = [faq["question"] for faq in faq_data]
        matches = get_close_matches(user_input.lower(), 
                                  [q.lower() for q in questions], 
                                  n=1, 
                                  cutoff=0.6)
        
        if not matches:
            return None
            
        # Find the original question with proper casing
        for question in questions:
            if question.lower() == matches[0]:
                return question
                
        return matches[0]
    except Exception as e:
        print(f"Error in matching: {str(e)}")
        return None

@app.route("/ask", methods=["POST"])
def ask():
    """Handle FAQ questions"""
    try:
        # Validate request
        if not request.is_json:
            return jsonify({
                "error": "Request must be JSON"
            }), 400

        data = request.get_json()
        
        if not data or "question" not in data:
            return jsonify({
                "error": "Please provide a question in the JSON body"
            }), 400

        user_input = data["question"]
        
        if not user_input or not isinstance(user_input, str):
            return jsonify({
                "error": "Question must be a non-empty string"
            }), 400

        # Find matching question
        best_question = get_best_match(user_input, faq_data)
        
        if best_question:
            for faq in faq_data:
                if faq["question"].lower() == best_question.lower():
                    return jsonify({
                        "question": best_question,
                        "answer": faq["answer"],
                        "match_confidence": "high" if best_question.lower() == user_input.lower() else "medium"
                    })

        return jsonify({
            "error": "No matching question found",
            "message": "Try rephrasing your question"
        }), 404

    except Exception as e:
        return jsonify({
            "error": "Internal server error",
            "message": str(e)
        }), 500

@app.route("/questions", methods=["GET"])
def list_questions():
    """Return list of all available questions"""
    try:
        questions = [faq["question"] for faq in faq_data]
        return jsonify({
            "questions": questions,
            "total": len(questions)
        })
    except Exception as e:
        return jsonify({
            "error": "Internal server error",
            "message": str(e)
        }), 500

if __name__ == "__main__":
    app.run(debug=True, host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


SystemExit: 1